# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
df = data.groupby(by=['CustomerID','ProductName'], as_index=False).Quantity.sum()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [51]:
pt = df.pivot_table(index ='ProductName' , columns = 'CustomerID', fill_value = 0)
pt

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube                0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp        0     0     0     0     0     0   
Appetizer - Mushroom Tart                0     0     0     0     0     0   
Appetizer - Sausage Rolls                0     0     0     0     0     0   
Apricots - Dried                         1     0     0     0     1     0   
...                                    ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                     0     0     0     0     0     0   
Yoghurt Tubes                            0     0     0     0     0     0   
Yogurt - Blueberry, 175 Gr               0     1     0     0     0     0   
Yogurt - French Vanilla                  1     0     0     1     0     0   
Zucchini - Yellow                        0     0     0     0     0     0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube             0     1     0     0  ...     0    25   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0  ...    25    25   
Appetizer - Mushroom Tart             0     1     0     0  ...    25     0   
Appetizer - Sausage Rolls             0     0     0     0  ...     0     0   
Apricots - Dried                      0     0     0     0  ...     0    25   
...                                 ...   ...   ...   ...  ...   ...   ...   
Yeast Dry - Fermipan                  0     0     0     0  ...     0     0   
Yoghurt Tubes                         0     0     2     0  ...     0     0   
Yogurt - Blueberry, 175 Gr            0     0     0     0  ...    25     0   
Yogurt - French Vanilla               2     0     0     1  ...     0     0   
Zucchini - Yellow                     0     0     0     0  ...     0     0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0    25   
Appetizer - Sausage Rolls             0     0    25    25    25     0    25   
Apricots - Dried                      0     0     0     0     0     0     0   
...                                 ...   ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                  0    25     0     0     0     0     0   
Yoghurt Tubes                         0     0     0     0     0    25     0   
Yogurt - Blueberry, 175 Gr            0    25     0     0     0     0     0   
Yogurt - French Vanilla              25     0     0     0     0     0     0   
Zucchini - Yellow                     0     0     0     0     0     0     0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube             0  
Appetizer - Mini Egg Roll, Shrimp     0  
Appetizer - Mushroom Tart             0  
Appetizer - Sausage Rolls             0  
Apricots - Dried                      0  
...                                 ...  
Yeast Dry - Fermipan                  0  
Yoghurt Tubes                         0  
Yogurt - Blueberry, 175 Gr            0  
Yogurt - French Vanilla              25  
Zucchini - Yellow                     0  

[452 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [26]:
s_matrix = pd.DataFrame(squareform(pdist(pt.T)), index = pt.column, columns = pt.column)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [64]:
s_matrix.index

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

In [68]:
s_matrix.iloc[:,[0]].drop(33).sort_values(by = 33).head(5)

CustomerID,33
CustomerID,
264,10.488088
3317,10.488088
3535,10.488088
2503,10.630146
3305,10.677078


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [40]:
similar_customers = s_matrix.iloc[:,[0]].drop(33).sort_values(by = 33).head(5).index

In [69]:
records = []
for index in similar_customers:
    records.append(data.loc[data['CustomerID']==index,:])
onefive = pd.concat(records)
onefive

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
230,92255,Brendan,Liu,1037077,270,Cheese Cloth No 100,24
392,92255,Brendan,Liu,3285561,153,Arizona - Green Tea,24
2875,92255,Brendan,Liu,4289376,409,Skirt - 29 Foot,24
3196,92255,Brendan,Liu,4932967,85,Cheese - Parmesan Cubes,24
4739,92255,Brendan,Liu,507732,252,Nantucket - Pomegranate Pear,24
...,...,...,...,...,...,...,...
65041,91777,Ramona,Cabrera,4813508,327,"Appetizer - Mini Egg Roll, Shrimp",24
65356,91777,Ramona,Cabrera,2466278,18,"Lamb - Whole, Fresh",24
65684,91777,Ramona,Cabrera,3127413,169,Water - Aquafina Vitamin,24
65820,91777,Ramona,Cabrera,4445712,353,Milk Powder,24


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [70]:
onesix = onefive.groupby(by = 'ProductName', 
                         as_index = False).Quantity.sum().sort_values(by = 'Quantity', 
                                                                      ascending = False)

In [71]:
onesix

,ProductName,Quantity
21,"Beef - Tenderlion, Center Cut",122
227,"Turnip - White, Organic",122
209,Squid U5 - Thailand,121
134,Muffin Chocolate Individual Wrap,98
159,Pastry - Butterscotch Baked,98
...,...,...
75,Crackers Cheez It,24
188,"Sauce - Gravy, Au Jus, Mix",24
187,Sardines,24
186,"Salsify, Organic",24


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [72]:
oneseven = pd.merge(onesix, pt[[('Quantity',    33)]], how = 'left', on = 'ProductName')

C:\Users\55119\AppData\Roaming\Python\Python37\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [73]:
oneseven = oneseven.rename(columns = {('Quantity',    33):33})

In [74]:
oneseven.head(5)

,ProductName,Quantity,33
0,"Beef - Tenderlion, Center Cut",122,0
1,"Turnip - White, Organic",122,0
2,Squid U5 - Thailand,121,0
3,Muffin Chocolate Individual Wrap,98,0
4,Pastry - Butterscotch Baked,98,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [87]:
def recommendations(df):
    pt = df.pivot_table(index='ProductName',columns='CustomerID',values='Quantity',fill_value = 0)
    similarities = pd.DataFrame(1/(1 + squareform(pdist(pt.T, metric = 'cosine'))), 
                         index=pt.columns, columns=pt.columns)
    rec = {}
    customers = df['CustomerID'].unique()
    for customer in customers:
        onefour = similarities[[customer]].drop(customer).sort_values(by=customer,ascending=False).head()
        records = []
        for index in onefour.index:
            records.append(df.loc[df['CustomerID']==index,:])
        onefive = pd.concat(records)
        onesix = onefive.groupby('ProductName')[['Quantity']].sum().sort_values(by='Quantity',ascending = False)
        oneseven = pd.merge(left = pt, right=onesix, how='left', left_on ='ProductName', right_on ='ProductName')
        o=oneseven.loc[oneseven[customer]==0,:][['Quantity']].sort_values(by = 'Quantity', ascending = False).head()
        r = {}
        for index in o.index:
            r[index] = o.at[index,'Quantity']
        rec[customer] = r
    return rec

In [81]:
euc = recommendations(data)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [77]:
pd.DataFrame(recommendations(data))

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
"Mushrooms - Black, Dried",15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,NaN
"Wine - Magnotta, Merlot Sr Vqa",14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chicken - Soup Base,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wine - Chardonnay South,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Milk - 1%,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Garbag Bags - Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cheese - Cambozola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Truffle Cups - Brown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oil - Safflower,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [80]:
cityblock = recommendations(data)

In [82]:
cityblock[33]

{'Oranges - Navel, 72': 3.0,
 'Chocolate - Dark': 3.0,
 'Pecan Raisin - Tarts': 3.0,
 'Bay Leaf': 3.0,
 'Wine - Charddonnay Errazuriz': 3.0}

In [83]:
euc[33]

{'Oranges - Navel, 72': 3.0,
 'Chocolate - Dark': 3.0,
 'Pecan Raisin - Tarts': 3.0,
 'Bay Leaf': 3.0,
 'Wine - Charddonnay Errazuriz': 3.0}

In [85]:
cityblock[61288]

{'Cheese - Boursin, Garlic / Herbs': 4.0,
 'Pork - Kidney': 4.0,
 'Bay Leaf': 3.0,
 'Wine - Ej Gallo Sierra Valley': 3.0,
 'Oranges - Navel, 72': 3.0}

In [86]:
euc[61288]

{'Cheese - Boursin, Garlic / Herbs': 4.0,
 'Pork - Kidney': 4.0,
 'Bay Leaf': 3.0,
 'Wine - Ej Gallo Sierra Valley': 3.0,
 'Oranges - Navel, 72': 3.0}

In [ ]:
# For cityblock and euclidean metrics the distances are very similar

In [89]:
cosine = recommendations(data)

In [90]:
cosine[77352]

{'Mangoes': 62.0,
 'Lettuce - Frisee': 58.0,
 'Zucchini - Yellow': 55.0,
 'Apricots Fresh': 55.0,
 'Juice - Apple Cider': 52.0}

In [91]:
euc[77352]

{'Juice - Lime': 3.0,
 'Bread Crumbs - Panko': 3.0,
 'Soup - Campbells Tomato Ravioli': 3.0,
 'Veal - Sweetbread': 3.0,
 'Sausage - Liver': 3.0}

In [92]:
cityblock[77352]

{'Juice - Lime': 3.0,
 'Bread Crumbs - Panko': 3.0,
 'Soup - Campbells Tomato Ravioli': 3.0,
 'Veal - Sweetbread': 3.0,
 'Sausage - Liver': 3.0}

In [ ]:
#Comparing the recommendations generated by the cosine distance we can observe that the way it is calculated is
#totally different the euclidean or the cityblock.